In [1]:
import janitor
import pandas as pd

In [2]:
RAW_DATA = "./arte_urbana_manual.xlsx"
OUTPUT_DATA = "../src/data.json"

In [3]:
df = pd.read_excel(
    RAW_DATA,
    # index_col="ID",
    verbose=True,
    dtype={"Data": "Int64"},
)

Reading sheet 0


In [4]:
df.head()

,ID,name,Autoria,Localizacao,Freguesia,Data,Tipologia,pic_url,Latitude,Longitude,Descricao
0,1,Sem título,Odeith,Troço IC17/Cril,Falagueira-Venda Nova,2010,Muro,http://geoportal.cm-amadora.pt/files_site_inte...,38.758635,-9.206684,«Acho que quem gosta de graffiti acaba sempre ...
1,2,Sem título,Dzenhart,Troço IC17/Cril,Encosta do Sol,2010,Muro,http://geoportal.cm-amadora.pt/files_site_inte...,38.759657,-9.201706,Mural realizado no âmbito do Concurso de Pintu...
2,3,Bob Marley,Odeith,Bairro da Cova da Moura,Águas Livres,2010,Muro,http://geoportal.cm-amadora.pt/files_site_inte...,38.746397,-9.213829,«Era mais influenciado pelo que vivia no dia-a...
3,4,Os verdadeiros heróis nunca serão esquecidos,Odeith,Rua Dr. Francisco Sousa Tavares,Águas Livres,2013,Hall of Fame (Extensão Mural),http://geoportal.cm-amadora.pt/files_site_inte...,38.741136,-9.216830,Odeith (2013). Mural localizado nas traseiras ...
4,5,Sem título,GVS Crew,Troço IC17/Cril,Falagueira-Venda Nova,2010,Muro,http://geoportal.cm-amadora.pt/files_site_inte...,38.750666,-9.211668,"GVS Crew (2010). Eko, membro da GVS Crew: “O m..."


In [5]:
df.shape

(110, 11)

http://geoportal.cm-amadora.pt/graffiti/Arte_Urbana/index.html

**Quotation marks**

- «
- »
- “
- ”

In [6]:
df["Descricao"].to_list()

['«Acho que quem gosta de graffiti acaba sempre por fazer quase o ritual todo.»\nPeça realizada no âmbito do Concurso de Pintura de Murais sobre Banda Desenhada, promovido pelas Estradas de Portugal e em parceria com a Câmara Municipal da Amadora.',
 'Mural realizado no âmbito do Concurso de Pintura de Murais sobre Banda Desenhada, promovido pelas Estradas de Portugal e em parceria com a Câmara Municipal da Amadora.',
 '«Era mais influenciado pelo que vivia no dia-a-dia.»\nMural localizado no Bairro da Cova da Moura, na freguesia da Damaia. Exibe uma figura mediática e vista como um exemplo na comunidade, o intérprete Bob Marley.',
 'Odeith (2013). Mural localizado nas traseiras da Escola Secundária D.João V, na Damaia. Este mural representa uma homenagem aos bombeiros, vítimas dos incêndios.',
 'GVS Crew (2010). Eko, membro da GVS Crew: “O meu interesse surgiu no ano de 1997, ano esse que se assistia a um grande boom da cultura hip-hop, desse boom surgem muitas crews e muitos writers.

In [ ]:
indent = 4

# df.to_json(OUTPUT_DATA, orient="records", force_ascii=False, indent=indent)

---